<a href="https://colab.research.google.com/gist/vrtmrz/37c3efd7842e49947aaaa7f665e5020a/deploy_couchdb_to_flyio_v2_with_swap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

History:
- 18, May, 2023: Initial.
- 19, Jun., 2023: Patched for enabling swap.
- 22, Aug., 2023: Generating Setup-URI implemented.
- 7, Nov., 2023: Fixed the issue of TOML editing.

In [ ]:
# Configurations
import os
os.environ['region']="nrt"
os.environ['couchUser']="alkcsa93"
os.environ['couchPwd']="c349usdfnv48fsasd"

In [ ]:
# Delete once (Do not care about `cannot remove './fly.toml': No such file or directory`)
!rm ./fly.toml

In [ ]:
# Installation
# You have to set up your account in here.
!curl -L https://fly.io/install.sh | sh
!/root/.fly/bin/flyctl auth signup

In [ ]:
# Generate server
!/root/.fly/bin/flyctl launch  --auto-confirm --generate-name --detach --no-deploy --region ${region}
!/root/.fly/bin/fly volumes create --region ${region} couchdata --size 2 --yes

In [ ]:
# Check the toml once.
!cat fly.toml

In [ ]:
# Modify the TOML and generate Dockerfile
!pip install mergedeep
from mergedeep import merge
import toml
fly = toml.load('fly.toml')
override = {
    "http_service":{
        "internal_port":5984
    },
    "build":{
        "dockerfile":"./Dockerfile"
    },
    "mounts":{
        "source":"couchdata",
        "destination":"/opt/couchdb/data"
    },
    "env":{
        "COUCHDB_USER":os.environ['couchUser'],
        "ERL_FLAGS":"-couch_ini /opt/couchdb/etc/default.ini /opt/couchdb/etc/default.d/ /opt/couchdb/etc/local.d /opt/couchdb/etc/local.ini /opt/couchdb/data/persistence.ini",
    }
}
out = merge(fly,override)
with open('fly.toml', 'wt') as fp:
    toml.dump(out, fp)
    fp.close()

# Make the Dockerfile to modify the permission of the ini file. If you want to use a specific version,  you should change `latest` here.
dockerfile = '''FROM couchdb:latest
RUN sed -i '2itouch /opt/couchdb/data/persistence.ini && chmod +w /opt/couchdb/data/persistence.ini && fallocate -l 512M /swapfile && chmod 0600 /swapfile && mkswap /swapfile && echo 10 > /proc/sys/vm/swappiness && swapon /swapfile && echo 1 > /proc/sys/vm/overcommit_memory' /docker-entrypoint.sh
'''
with open("./Dockerfile","wt") as fp:
    fp.write(dockerfile)
    fp.close()

!echo ------
!cat fly.toml
!echo ------
!cat Dockerfile

In [ ]:
# Configure password
!/root/.fly/bin/flyctl secrets set COUCHDB_PASSWORD=${couchPwd}

In [ ]:
# Deploy server
# Be sure to shutdown after the test.
!/root/.fly/bin/flyctl deploy --detach --remote-only
!/root/.fly/bin/flyctl status

In [ ]:
import subprocess, json
result = subprocess.run(["/root/.fly/bin/flyctl","status","-j"], capture_output=True, text=True)
if result.returncode==0:
    hostname = json.loads(result.stdout)["Hostname"]
    os.environ['couchHost']="https://%s" % (hostname)
    print("Your couchDB server is https://%s/" % (hostname))
else:
    print("Something occured.")


In [ ]:
# Finish setting up the CouchDB
# Please repeat until the request is  completed without error messages
# i.e., You have to redo this block while "curl: (35) OpenSSL SSL_connect: Connection reset by peer in connection to xxxx" is showing.
#
# Note: A few minutes might be required to be booted.
!curl -X POST "${couchHost}/_cluster_setup" -H "Content-Type: application/json" -d "{\"action\":\"enable_single_node\",\"username\":\"${couchUser}\",\"password\":\"${couchPwd}\",\"bind_address\":\"0.0.0.0\",\"port\":5984,\"singlenode\":true}"  --user "${couchUser}:${couchPwd}"

In [ ]:
# Please repeat until all lines are completed without error messages
!curl -X PUT "${couchHost}/_node/nonode@nohost/_config/chttpd/require_valid_user" -H "Content-Type: application/json" -d '"true"' --user "${couchUser}:${couchPwd}"
!curl -X PUT "${couchHost}/_node/nonode@nohost/_config/chttpd_auth/require_valid_user" -H "Content-Type: application/json" -d '"true"' --user "${couchUser}:${couchPwd}"
!curl -X PUT "${couchHost}/_node/nonode@nohost/_config/httpd/WWW-Authenticate" -H "Content-Type: application/json" -d '"Basic realm=\"couchdb\""' --user "${couchUser}:${couchPwd}"
!curl -X PUT "${couchHost}/_node/nonode@nohost/_config/httpd/enable_cors" -H "Content-Type: application/json" -d '"true"' --user "${couchUser}:${couchPwd}"
!curl -X PUT "${couchHost}/_node/nonode@nohost/_config/chttpd/enable_cors" -H "Content-Type: application/json" -d '"true"' --user "${couchUser}:${couchPwd}"
!curl -X PUT "${couchHost}/_node/nonode@nohost/_config/chttpd/max_http_request_size" -H "Content-Type: application/json" -d '"4294967296"' --user "${couchUser}:${couchPwd}"
!curl -X PUT "${couchHost}/_node/nonode@nohost/_config/couchdb/max_document_size" -H "Content-Type: application/json" -d '"50000000"' --user "${couchUser}:${couchPwd}"
!curl -X PUT "${couchHost}/_node/nonode@nohost/_config/cors/credentials" -H "Content-Type: application/json" -d '"true"' --user "${couchUser}:${couchPwd}"
!curl -X PUT "${couchHost}/_node/nonode@nohost/_config/cors/origins" -H "Content-Type: application/json" -d '"app://obsidian.md,capacitor://localhost,http://localhost"' --user "${couchUser}:${couchPwd}"

Now, our CouchDB has been surely installed and configured. Cheers!

In the steps that follow, create a setup-URI.

This URI could be imported directly into Self-hosted LiveSync, to configure the use of the CouchDB which we configured now.

In [ ]:
# Database config
import random, string

def randomname(n):
   return ''.join(random.choices(string.ascii_letters + string.digits, k=n))

# The database name
os.environ['database']="obsidiannote"
# The passphrase to E2EE
os.environ['passphrase']=randomname(20)

print("Your database:"+os.environ['database'])
print("Your passphrase:"+os.environ['passphrase'])

In [ ]:
# Install deno for make setup uri
!curl -fsSL https://deno.land/x/install/install.sh | sh

In [ ]:
# Fetch module for encrypting a Setup URI
!curl -o encrypt.ts https://gist.githubusercontent.com/vrtmrz/f9d1d95ee2ca3afa1a924a2c6759b854/raw/d7a070d864a6f61403d8dc74208238d5741aeb5a/encrypt.ts

In [ ]:
# Make buttons!
from IPython.display import HTML
result = subprocess.run(["/root/.deno/bin/deno","run","-A","encrypt.ts"], capture_output=True, text=True)
text=""
if result.returncode==0:
  text = result.stdout.strip()
  result = HTML(f"<button onclick=navigator.clipboard.writeText('{text}')>Copy setup uri</button><br>Importing passphrase is `welcome`. <br>If you want to synchronise in live mode, please apply a preset after setup.)")
else:
  result = "Failed to encrypt the setup URI"
result